## Goal
We want to explore how better is the performance of the system of generating a draft with a multiple step process rather than a single step process

### Initial Variables

In [1]:
import sys 
import os
import json_tricks as json
import time
from dotenv import load_dotenv
load_dotenv('../../backend/.env')
sys.path.append("../../")
from backend.LLM.AnythingLLM_client import AnythingLLMClient
from backend.Generater import Generater
from backend.LLM.OllamaLLM import OllamaAI
from backend.database.schemas import Email


anyllm_client = AnythingLLMClient("http://localhost:3001/api", "3WMNAPZ-GYH4RBE-M67SR00-7Y7KYEF")
ollama_client = OllamaAI('http://localhost:11434', 'llama3:instruct')
generator = Generater(ollama_client, anyllm_client)
path_to_enquiry = "../../dataset/fake_email_1.json"
with open(path_to_enquiry, "r") as f:
    enquiry = json.load(f)

enquiry_text = enquiry["email"]
subject = enquiry_text.split("\n\n")[0]
body = '\n'.join(enquiry_text.split("\n\n")[1:])

Performing health check on http://localhost:11434
Ollama is running
Health check passed


### Single Step Process

In [2]:

def single_shot_prompt(email: Email):
    prompt = f""" You are David Frischer, the program administrator for the UCL Software Engineering program.
            You have received the following enquiry from a student:
            Subject: {email.subject}
            Body: {email.body}
            
           
            Draft a response to the student's email.
            """
    workspace_slug =  anyllm_client.get_workspace_slug("General")
    thread = anyllm_client.new_thread(workspace_slug, "tmp-thread")
    thread_slug = thread["slug"]
    start_time = time.time()
    response = anyllm_client.chat_with_thread(workspace_slug, thread_slug, prompt)
    total_time = time.time() - start_time
    anyllm_client.delete_thread(workspace_slug, thread_slug)
    return response, total_time

### Response format

## Mutiple Step Process

In [3]:
def multi_step_process(email:Email):
    start_time = time.time()
    response = generator.single_run_reply_to_email(email)
    response_time = time.time() - start_time
    
    return response, response_time
    

## Generate for multiple drafts

In [5]:

path_to_enquiries = "../../dataset"

counter = 0
results_folder = "./results"
os.makedirs(results_folder, exist_ok=True)

already_processed = set()
for results_file in os.listdir(results_folder):
    if results_file.endswith(".json"):
        already_processed.add(results_file.replace("results_", ""))
        
for enquiry_path in os.listdir(path_to_enquiries):
    if enquiry_path in already_processed:
        print(f"Already processed {enquiry_path}")
        continue
    if "fake" not in enquiry_path and not enquiry_path.endswith(".json"):
        print(f"Skipping {enquiry_path}")
        continue
      
    filepath = os.path.join(results_folder, f"results_{enquiry_path}")
    results = {}
    try:
        if not enquiry_path.endswith(".json") or not "fake" in enquiry_path:
            continue
        print(f"Processing {enquiry_path}")
        with open(os.path.join(path_to_enquiries, enquiry_path), "r") as f:
            enquiry = json.load(f)
            
        email = enquiry["email"]
        if "subject:" in email.lower():
            subject = email.split("\n")[0].replace("Subject:", "").strip()
            body = "\n".join(email.split("\n")[1:])
        else:
            subject = "Subject not found"
            body = email
        
        email = Email(subject=subject, body=body, id=counter)
        counter += 1
        
        response_single, response_time_single = single_shot_prompt(email)
        response_multi, response_time_multi = multi_step_process(email)
        
        results = {"single_shot": {"response": response_single, "response_time": response_time_single},
                    "multi_step": {"response": response_multi, "response_time": response_time_multi}}

        with open(filepath, "w") as f:
            json.dump(results, f)
            
    except Exception as e:
        print(f"Error processing {enquiry_path}")
        print(e)
        continue    

print("Done")


Already processed fake_email_10.json
Already processed fake_email_undergrad_13.json
Already processed fake_email_undergrad_6.json
Already processed fake_email_3.json
Already processed fake_email_2.json
Already processed fake_email_undergrad_7.json
Already processed fake_email_undergrad_12.json
Already processed fake_email_11.json
Already processed fake_email_undergrad_0.json
Already processed fake_email_undergrad_19.json
Already processed fake_email_5.json
Already processed fake_email_16.json
Already processed fake_email_undergrad_15.json
Already processed fake_email_9.json
Already processed fake_email_8.json
Already processed fake_email_undergrad_14.json
Already processed fake_email_17.json
Skipping __pycache__
Already processed fake_email_4.json
Already processed fake_email_undergrad_18.json
Already processed fake_email_undergrad_1.json
Already processed fake_email_7.json
Skipping bank_questions.py
Already processed fake_email_undergrad_2.json
Already processed fake_email_18.json
Pro

[ERROR] Expecting value: line 1 column 1 (char 0)
[ERROR] Expecting value: line 1 column 1 (char 0)


Creating new thread with name: new_thread in workspace: general
Total Sim Distance: 0.3227524310350418
Total Sim Distance: 0.3550126403570175
['Question: Could you please let me know when I can expect to receive a decision on my application?\nAnswer: According to the provided contexts from University College London\'s website, it seems that the university uses a "gathered field" approach in assessing applications. This means they need to wait until they have enough comparable applications to make a final decision.\n\nAs mentioned in Context 0 and Context 1, the university aims to make decisions by the end of April but are working hard to send decisions sooner. It\'s also stated that it may take some time before they can give you a final decision due to the large number of well-qualified applicants.\n\nIn your case, since you\'ve just submitted your application for the undergraduate program in Computer Science, I would advise waiting until the university has processed enough comparable 

[ERROR] Expecting value: line 1 column 1 (char 0)
[ERROR] Expecting value: line 1 column 1 (char 0)


Processing fake_email_14.json
Creating new thread with name: tmp-thread in workspace: general


...

[ERROR] Expecting value: line 1 column 1 (char 0)
[ERROR] Expecting value: line 1 column 1 (char 0)


Creating new thread with name: new_thread in workspace: general
Total Sim Distance: 0.269557848572731
Total Sim Distance: 0.3257996141910553
Total Sim Distance: 0.3105001002550125
["Question: When can I expect to receive a decision on my fee status?\nAnswer: Hello Lewis!\n\nThank you for reaching out to us with your query. As an international student applying for the BSc in Computer Science program at University College London (UCL), we're happy to help you with any questions or concerns.\n\nRegarding your question about when you can expect to receive a decision on your fee status, we aim to provide you with the outcome of your fee status assessment before you need to make your Firm and Insurance decisions. However, in some cases it may not be possible to do this if additional information is required from you to inform our decision.\n\nIf further information is needed, we'll email you a Fee Status Questionnaire for you to complete. We appreciate that waiting for a university decision c

[ERROR] Expecting value: line 1 column 1 (char 0)
[ERROR] Expecting value: line 1 column 1 (char 0)


Processing fake_email_6.json
Creating new thread with name: tmp-thread in workspace: general


KeyboardInterrupt: 